In [ ]:
import warnings

warnings.filterwarnings("ignore")

# Grid

At the top of the Equi7Grid entities is the grid itself, which is a collection of tiling systems sharing a tiling concept, but differ in their projection (however, the projection should be from the same family). The Equi7Grid for instance is a collection of seven continental equidistant azimuthal projections. Another example would be the UTM projection with 60 cylindrical projections for 6 degree steps around the globe.

The standard realisation of the Equi7Grid has seven continental tiling schemes: Africa ("AF"), Antarctica ("AN"), Asia ("AS"), Europe ("EU"), North America ("NA"), Oceania ("OC"), and South America ("SA"). Each continent is a consistent tiling system with three tiling levels by default: T1 (100km tile size), T3 (300km tile size), and T6 (600km tile size).


## Initialisation

The `equi7grid` projection provides four user-facing functions to create an `Equi7Grid` object: 

- `get_standard_equi7grid`: returns the standard Equi7Grid as defined above
- `get_user_equi7grid`: returns a user-defined version of the Equi7Grid to allow for other tilings 
- `get_equi7grid_from_grid_def`: returns an Equi7Grid instance based on a grid definition in a JSON file (can be used to share a specific definition with other people)
- `get_equi7grid_from_file`: returns an Equi7Grid instance based on an instance definition in a JSON file, i.e., everything is set in stone, e.g. the sampling cannot be changed anymore (can be used to share a specific instance with other people)

Lets take a look at the first two methods, which do not have require any additional files. One important argument for these functions is the `sampling`, which can be specified in several ways:

- `sampling=500`: all tiling levels will use a sampling of 500m.
- `sampling={"T6": 500, "T3": 20, "T1": 10}`: each tiling level has a specific sampling
- `sampling={"T3": 40}`: there is only one tiling level with the specified sampling, all others will be discarded

Now we know eveything to create our first `Equi7Grid` instance:

In [ ]:
from equi7grid import get_standard_equi7grid

e7grid = get_standard_equi7grid(500)
e7grid

There are two optional arguments: `buffered` and `continent_order`. `buffered` is a boolean flag (defaults to false) defining if a buffered version of the projection zones should be used or not. These special versions of the projection zones have a 100km buffer over land to be able to fully represent certain countries in one projection/tiling system. Note that this means that a global coordinate, e.g. a longitude latitude coordinate, might intersect with multiple projection zones. As an example, when reprojecting a geographic coordinate to the Equi7Grid, multiple Equi7Grid coordinates might be returned.

This is why there is a second argument `continent_order` to be able to control the order of the returned coordinates or tiling systems. `continent_order` is a list of Equi7Grid continent identifiers specifying the desired order of the returned values and does not need to be complete. If a subset is specified, then other continents are not touched or returned allowing to speed up initialisation and queries. 

Another way to create an Equi7Grid instance is with `get_user_equi7grid`. This method provides more flexibility in terms of introducing a user-specific tiling scheme, e.g. "T2" with a tile size of 200km. For this we need the `RegularTilingDefinition` class from `pytileproj` and the desired tile size/shape associated with the tiling ID/name. The tiling definition is then specified as a dictionary with the tiling levels as keys and the `RegularTilingDefinition` instances as values.

In [ ]:
from pytileproj import RegularTilingDefinition

from equi7grid import get_user_equi7grid

tiling_defs = {1: RegularTilingDefinition(name="T2", tile_shape=200_000)}

e7grid = get_user_equi7grid(500, tiling_defs)
e7grid

### Export and import grids

Another possibility is to start from a more generic regular grid definition. As an example, the core tiling system definitions of the `Equi7Grid` object can be exported to a JSON file via

In [ ]:
from pathlib import Path

grid_def_path = Path("my_grid_def.json")
e7grid.to_grid_def(grid_def_path)

If we open it, it looks like this:

In [ ]:
import json
import pprint

with grid_def_path.open() as f:
    grid_def = json.load(f)

pprint.pprint(grid_def)  # noqa: T203

Now we can use this file in `get_equi7grid_from_grid_def` to re-create the object by specifying the desired sampling. Like this it is convenient to share a grid definition with other people.

In [ ]:
from equi7grid import get_equi7grid_from_grid_def

e7grid = get_equi7grid_from_grid_def(grid_def_path, 500)
e7grid

If we want to be more explicit, there is also the option to export all class properties to a JSON file. This includes the sampling, thus allowing to exactly replicate the object, from which the JSON file was created.

In [ ]:
grid_path = Path("my_grid.json")
e7grid.to_file(grid_path)

Now the JSON file looks like this:

In [ ]:
with grid_path.open() as f:
    grid = json.load(f)

pprint.pprint(grid)  # noqa: T203

And the object can be re-created in the same manner:

In [ ]:
from equi7grid import get_equi7grid_from_file

e7grid = get_equi7grid_from_file(grid_path)
e7grid

### Supported samplings

If you are not sure, what samplings you can use for your grid design, you can fetch them directly via the `allowed_samplings` method with the desired tile size as an argument:

In [ ]:
from equi7grid import Equi7Grid

Equi7Grid.allowed_samplings(300)

## Tiling system access

[Equi7 tiling systems](https://tuw-geo.github.io/Equi7Grid/latest/guides/tiling_system.html) can then be accessed with a dot on the grid instance (works with autocompletion):

In [ ]:
e7grid.AF

or equivalently,

In [ ]:
e7grid["AF"]

In case you want to know which tiling system(s) to use for a certain location, then you can use `get_systems_from_coord` or `get_systems_from_lonlat`.

In [ ]:
e7grid.get_systems_from_lonlat(16, 48)

## Geographic coordinate transformation

An `Equi7Grid` instance also allows to transform geographic coordinates to grid coordinates. 

In [ ]:
e7grid.lonlat_to_xy(16, 48)

Usually, there is a unique result. But in case projection zones of different tiling systems overlap, coordinates are returned for each intersecting tiling system.

In [ ]:
e7grid = get_standard_equi7grid(500, buffered=True)
e7grid.lonlat_to_xy(50, 58)

The order is defined by `continent_order` during initialisation, which can be only one continent to work with a single tiling system.

In [ ]:
e7grid = get_standard_equi7grid(500, buffered=True, continent_order=["AS"])
e7grid.lonlat_to_xy(50, 58)

## Tile queries

The [tile documentation](https://tuw-geo.github.io/Equi7Grid/latest/guides/tile.html#properties) explains the structure of an Equi7 tile name. If we know the tile name of a specific tile we are interested in, we can create the tile with

In [ ]:
e7grid = get_standard_equi7grid(500)

e7grid.get_tile_from_name("AS_E012N006T6")

[Tile queries](https://tuw-geo.github.io/Equi7Grid/latest/guides/tile.html#tiles) are already available for an Equi7 tiling system instance, but there is also the option to perform spatial tile queries at a grid level. All functions work in the same manner, with a `cover_land` flag to filter for tiles covering land and a generator object as a return value. With `get_tiles_in_geog_bbox` you can retrieve all tiles accross all continents intersecting with the given geographical bounding box.

In [ ]:
e7tiles = e7grid.get_tiles_in_geog_bbox(
    bbox=(0, 30, 10, 40), tiling_id="T6", cover_land=True
)
[e7tile.name for e7tile in e7tiles]

:::{important}
For a geographical bounding box the order of longitude coordinates is important. As an example: `(-170, 20, 150, 30)` is a large bounding box going from -170° west to 150° east and `(150, 20, -170, 30)` is a small bounding box crossing the antimeridian.
:::

`get_tiles_in_geom` is more generic and allows to provide any geometry for performing a tile query.

In [ ]:
import shapely
from pytileproj import GeogGeom

spitzbergen_points = [
    (8.391827331539572, 77.35762113396143),
    (25.43098663332705, 75.61353436967198),
    (40.50119498304080, 79.73786853853339),
    (16.87007957357446, 81.59290885863483),
]
spitzbergen_geom = GeogGeom(geom=shapely.Polygon(spitzbergen_points))

e7tiles = e7grid.get_tiles_in_geom(spitzbergen_geom, tiling_id="T1")
[(e7tile.name, e7tile.covers_land, e7tile.geotrans) for e7tile in e7tiles]

:::{note}
All geometries passed to `equi7grid` functions and class methods are segmentised to ensure an adequate representation of the geometry in a different projection system. A maximum segment length is defined as
- 0.1 degree for geographic projections
- 10 kilometres for metric projections
:::

In [ ]:
grid_path.unlink(missing_ok=True)
grid_def_path.unlink(missing_ok=True)